In [66]:
MODEL_PATH = 'bert_chinese_doc_pair_ce_classifier'

In [67]:
from sentence_transformers import CrossEncoder
from ast import literal_eval
import pandas as pd

In [68]:
bert_model = CrossEncoder(MODEL_PATH)

In [69]:
def correctly_read_csv(fname):
    prep_df = pd.read_csv(fname, converters={"tokens_rep": literal_eval, "tokens": literal_eval, "reference": literal_eval})
    return prep_df

In [9]:
test_df = correctly_read_csv("public_test_data_processed.csv")

In [10]:
test_df.head()

,doc_index,raw_text,tokens_rep,tokens
0,1053,近來氣候高溫多濕相當適合草莓苗期炭疽病發生籲請農友注意防治適期苗栗區農改場防檢局及田邊好幫手...,"[近來, 氣候, 高溫, 多, 濕, 相當, 適合, 草莓苗期, 苗期, 炭疽病, 發生, ...","[近來, 氣候, 高溫, 多, 濕, 相當, 適合, 草莓苗期, 炭疽病, 發生, 籲請, ..."
1,1084,白粉病好發期呼籲請農友加強防治工作雲林縣政府防檢局及田間好幫手關心您白粉病主要出現於秋季至隔...,"[白粉病, 好, 發期, 呼籲, 請, 農友, 加強, 防治, 工作, 雲林, 縣, 政府,...","[白粉病, 好, 發期, 呼籲, 請, 農友, 加強, 防治, 工作, 雲林, 縣, 政府,..."
2,538,水稻即將進入抽穗期氣候持續陰晴不定適合穗稻熱病的發生臺中區農業改良場籲請農友掌握防治時機減少...,"[水稻, 即將, 進入, 抽穗期, 氣候, 持續, 陰晴, 不定, 適合, 穗稻, 熱病, ...","[水稻, 即將, 進入, 抽穗期, 氣候, 持續, 陰晴, 不定, 適合, 穗稻, 熱病, ..."
3,1251,請宣導蕉農於蕉株抽穗期間適時做好花薊馬防治作業俾確保生產外觀優良品質香蕉香蕉花薊馬蟲口整年均...,"[請, 宣導, 蕉農, 於, 蕉株, 抽穗, 期間, 適時, 做好, 花薊馬, 防治, 作業...","[請, 宣導, 蕉農, 於, 蕉株, 抽穗, 期間, 適時, 做好, 花薊馬, 防治, 作業..."
4,909,發佈水象鼻蟲螟蟲及縱捲葉蟲發生警報水稻病蟲害發生警報中華民國九十三年八月六日第二號行政院農業...,"[發佈, 水象, 鼻蟲, 螟蟲, 及, 縱稻稻, 縱捲, 葉蟲, 發生, 警報, 水稻, 病...","[發佈, 水象, 鼻蟲, 螟蟲, 及, 縱捲, 葉蟲, 發生, 警報, 水稻, 病蟲害, 發..."


In [12]:
def remove_n_t_loop(strings):
    processed_strings = []
    for string in strings:
        string = string.replace('\n','')
        string = string.replace('\t','')
        string = string.replace('\r','')
        processed_strings.append(string)
    return processed_strings

In [13]:
test_df['raw_text'] = remove_n_t_loop(test_df['raw_text'])

In [70]:
bert_model.predict([[test_df['raw_text'][0], test_df['raw_text'][1]]])

array([0.00092127], dtype=float32)

In [15]:
test_df.raw_text[0]

'近來氣候高溫多濕相當適合草莓苗期炭疽病發生籲請農友注意防治適期苗栗區農改場防檢局及田邊好幫手關心您近來氣候高溫多濕相當適合草莓苗期炭疽病發生籲請農友注意防治適期防治應注意事項:(1)請加強母株的防治若育苗區母株已呈現萎凋狀請徹底清除勿留在園區(2)去除老葉之傷口為炭疽病菌侵入冠部之重要關鍵老葉去除後應施用藥劑以防病菌感染育苗區之葉片走蔓若有炭疽病斑請將較嚴重之葉片或走蔓去除並於24小時內配合藥劑之噴施(3)噴施藥劑時須特別注意冠部亦應有足夠之劑量因冠部若腐壞會造成植株萎凋死亡農友如果有任何技術上的疑問可以直接洽詢苗栗區農業改良場諮詢專線037-236583或病蟲害診斷專線037-236619或各鄉鎮市公所農會防治藥劑及方法(1)53腐絕快得寧可濕性粉劑1,200倍(2)24.9%待克利乳劑3,000倍(3)24.9%待克利水懸劑3,000倍(4)23.6%百克敏乳劑3,000倍'

In [18]:
#embeddings = bert_model.encode(test_df['raw_text'], batch_size=4)

In [19]:
#len(embeddings)

421

In [22]:
embeddings.shape

(421, 768)

In [23]:
embeddings[0].shape

(768,)

In [71]:
THRESH = 0.00184854446

In [75]:
def get_all_pair_scores(doc_ind_list, texts):
    all_pair_scores = []
    for i in range(len(texts)):
        for j in range(len(texts)):
            if i != j:
                prob = bert_model.predict([[texts[i], texts[j]]])[0]
                all_pair_scores.append([doc_ind_list[i], doc_ind_list[j], prob])
    return all_pair_scores

In [73]:
def get_pairs_meet_thresh(all_pairs):
    all_pairs_meet_thresh = []
    for lst in all_pairs:
        if lst[-1] > THRESH:
            all_pairs_meet_thresh.append(lst[:2])
    return all_pairs_meet_thresh

In [ ]:
all_scores = get_all_pair_scores(test_df.doc_index, test_df.raw_text)

In [58]:
from sklearn.metrics.pairwise import cosine_similarity

In [59]:
def get_all_similar_pairs(doc_ind_list, embeddings):
    doc_pairs_meet_thresh = []
    for i in range(len(embeddings)):
        for j in range(len(embeddings)):
            if i != j:
                similarity = cosine_similarity([embeddings[i]],
                                               [embeddings[j]])[0][0]
                if similarity >= THRESH:
                    doc_pairs_meet_thresh.append([doc_ind_list[i], doc_ind_list[j]])
    return doc_pairs_meet_thresh

In [60]:
similar_pairs = get_all_similar_pairs(test_df['doc_index'], embeddings)

In [61]:
len(similar_pairs)

670

In [62]:
import csv

In [63]:
def write_all_pairs_ids_to_csv(pairs, fname):
    with open(fname, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['Test', 'Reference'])
        for pair in pairs:
            writer.writerow(pair)

In [64]:
fname = 'bert_test_99thresh.csv'

In [65]:
write_all_pairs_ids_to_csv(similar_pairs, fname)